In [ ]:
#import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import model_from_json
from PIL import Image, ImageChops, ImageEnhance
import PIL
import os
import itertools
from tqdm import tqdm
tf.__version__

In [ ]:
def ELA(img_path, quality=90):
    TEMP = 'ela_' + 'temp.jpg'
    SCALE = 10
    original = Image.open(img_path)
    diff=""
    try:
        original.save(TEMP, quality=90)
        temporary = Image.open(TEMP)
        diff = ImageChops.difference(original, temporary)
        
    except:
        
        original.convert('RGB').save(TEMP, quality=90)
        temporary = Image.open(TEMP)
        diff = ImageChops.difference(original.convert('RGB'), temporary)
        
       
    d=diff.load()
    WIDTH, HEIGHT = diff.size
    for x in range(WIDTH):
        for y in range(HEIGHT):
            d[x, y] = tuple(k * SCALE for k in d[x, y])
#     save_path = dataset_path +'ELA_IMAGES/'
#     diff.save(save_path+'diff.png')
    return diff

In [ ]:
dataset_path="../input/casia-20-image-tampering-detection-dataset/CASIA2/"
path_original = 'Au/'
path_tampered = 'Tp/'
# path_mask='CASIA 2 Groundtruth/'
total_original = os.listdir(dataset_path+path_original)
total_tampered = os.listdir(dataset_path+path_tampered)
# total_mask=os.listdir(dataset_path+path_mask)


In [ ]:
pristine_images = []
for i in total_original:
    pristine_images.append(dataset_path+path_original+i)
fake_images = []
for i in total_tampered:
    fake_images.append(dataset_path+path_tampered+i)

In [ ]:
len(total_tampered),len(fake_images)

In [ ]:
image_size = (224,224)
output_path='./'

In [ ]:
# os.rmdir(output_path+"resized_images/fake_images/")
# os.rmdir(output_path+"resized_images/pristine_images/")
# os.rmdir(output_path+"resized_images/")

In [ ]:
output_path='./'
if not os.path.exists(output_path+"resized_images/"):
#     os.makedirs(output_path+"resized_images/fake_masks/")
    os.makedirs(output_path+"resized_images/fake_images/")
    os.makedirs(output_path+"resized_images/pristine_images/")
    height = 224
    width = 224
#     p2=output_path+"resized_images/fake_masks/"
    p1=output_path+"resized_images/fake_images/"
    p3=output_path+"resized_images/pristine_images/"
    j=0
    for fake_image in tqdm(total_tampered):
        try:
            if(j%3):
                j+=1
                continue
            img=Image.open(dataset_path+path_tampered + fake_image).convert("RGB")
            img = img.resize((height, width), PIL.Image.ANTIALIAS)
            img.save(p1+fake_image)
            j+=1
        except:
            print("Encountered Invalid File : ",fake_image)
        
    j=0
    for pristine_image in tqdm(total_original):
        try:
            if(j%3):
                j+=1
                continue
            img=Image.open(dataset_path+path_original + pristine_image).convert("RGB")
            img = img.resize((height, width), PIL.Image.ANTIALIAS)
            img.save(p3+pristine_image)
            j+=1
        except:
            print("Invalid File : " ,pristine_image)
        
        
        
else:
    print('images resized,path exists')

In [ ]:
resized_fake_image_path=output_path+"resized_images/fake_images/"
resized_pristine_image_path=output_path+"resized_images/pristine_images/"
resized_fake_image=os.listdir(resized_fake_image_path)
resized_pristine_image=os.listdir(resized_pristine_image_path)

In [ ]:
len(resized_pristine_image)
# os.rmdir(ela_real)
# os.rmdir(ela_fake)
# os.rmdir('ELA_IMAGES/')

In [ ]:
ela_images_path=output_path+'ELA_IMAGES/'
ela_real=ela_images_path+'Au/'
ela_fake=ela_images_path+'Tp/'
if not os.path.exists(ela_images_path):
    os.makedirs(ela_images_path)
    os.mkdir(ela_real)
    os.mkdir(ela_fake)
    j=0
    for i in tqdm(resized_fake_image):
        ELA(resized_fake_image_path+i).save(ela_fake+i)
        j+=1
        if(j==1500):
            break
    j=0
    for i in tqdm(resized_pristine_image):
        ELA(resized_pristine_image_path+i).save(ela_real+i)
        j+=1
        if(j==1500):
            break
else:
    print('Images are already converted to ELA')

In [ ]:
X=[]
Y=[]
j=0
for file in tqdm(os.listdir(ela_real)):
    img=Image.open(ela_real+file)
    img=np.array(img)
    X.append(img)
    Y.append(0)
    j+=1
    if(j==1500):
        break
j=0
for file in tqdm(os.listdir(ela_fake)):
    img=Image.open(ela_fake+file)
    img=np.array(img)
    X.append(img)
    Y.append(1)
    j+=1
    if(j==1500):
        break

In [ ]:
X=np.array(X)
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
x_train, x_dev, y_train, y_dev = train_test_split(X, Y, test_size=0.2, random_state=133,shuffle=True)
y_train=to_categorical(y_train,2)
y_dev=to_categorical(y_dev,2)

In [ ]:
from tensorflow.keras.applications import MobileNetV2,VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D,BatchNormalization,Dropout,MaxPooling2D
from tensorflow.keras.regularizers import l1,l2,l1_l2

base_model=MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')
for layer in base_model.layers:
    layer.trainable=False
x=base_model.output
x=Conv2D(1024,(3,3),padding='same',activation='relu')(x)
x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
x=Dense(1024,activation='relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
x=Dropout(0.3)(x)
x=Dense(16,activation='relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
x=Dense(2,activation='softmax')(x)
model=Model(base_model.input,x)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
# base=MobileNetV2()

In [ ]:
epochs = 100
batch_size = 20

In [ ]:
import keras
# if not os.path.exists('./model_checkpoints'):
#     os.makedirs('./model_checkpoints')
# # define callbacks for learning rate scheduling and best checkpoints saving
# filepath = './model_checkpoints/image_tampering_classification_ela.h5'
# checkpoint = keras.callbacks.ModelCheckpoint(filepath,monitor='val_accuracy',save_best_only=True,verbose=1)

early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=13,verbose=1,restore_best_weights=True)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.22, patience = 6, verbose = 1, 
                                              min_delta = 0.0001,min_lr=0.0001)

In [ ]:
hist = model.fit(x_train,y_train,
                 epochs = epochs,
                validation_data = (x_dev,y_dev),
                callbacks = [early_stop,reduce_lr],
                verbose=1,shuffle=True)

In [ ]:
# model.save('/kaggle/working/model_casia_run1.h5')
# model.save_weights('/kaggle/working/model_casia_run1.h5')

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(hist.history['loss'], color='b', label="Training loss")
ax[0].plot(hist.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(hist.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
feature_extractor=base_model.predict(x_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_training = features #This is our X input to RF

#RANDOM FOREST
#from sklearn.ensemble import RandomForestClassifier
#model = RandomForestClassifier(n_estimators = 50, random_state = 42)

# Train the model on training data
y_train_label=np.where(y_train==1)[1]

#XGBOOST
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(X_for_training, y_train_label) #For sklearn no one hot encoding

In [ ]:
import seaborn as sns
X_test_feature = base_model.predict(x_dev)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

#Now predict using the trained RF model. 
prediction = model.predict(X_test_features)
#Inverse le transform to get original label back. 
# prediction = le.inverse_transform(prediction)
y_test=np.where(y_dev==1)[1]
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction))

#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, prediction)
#print(cm)
sns.heatmap(cm, annot=True)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    


In [ ]:
pred=model.predict(X_test_features)

In [ ]:
# pred[0:20]

In [ ]:
# print("Loss of the model is - " , model.evaluate(X_test_features,y_test)[0])
# print("Accuracy of the model is - " , model.evaluate(X_test_features,y_test)[1]*100 , "%")
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(x_dev)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_dev,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(2))

Prediction

In [ ]:
class_names = ['real','fake']

In [ ]:
real_image_path = '../input/casia-20-image-tampering-detection-dataset/CASIA2/Tp/Tp_D_CNN_M_B_nat10139_nat00059_11949.jpg'
image = ELA(real_image_path)
image=image.resize((224, 224), PIL.Image.ANTIALIAS)
image=np.array(image)
image = image.reshape(-1, 224, 224, 3)
# image=np.array(image)
y_pred = model.predict(image)
y_pred_class = np.argmax(y_pred, axis = 1)[0]
print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')


In [ ]:
fake_image_path = '../input/casia-20-image-tampering-detection-dataset/CASIA2/Au/Au_ani_00022.jpg'
image = ELA(fake_image_path)
image=image.resize((224, 224), PIL.Image.ANTIALIAS)
image=np.array(image)
image = image.reshape(-1, 224, 224, 3)
# image=np.array(image)
y_pred = model.predict(image)
y_pred_class = np.argmax(y_pred, axis = 1)[0]
print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
total,correct=0,0

In [ ]:
for i in range(len(x_dev)):
    image=x_dev[i]
    image=np.array(image)
    image = image.reshape(-1, 224, 224, 3)
    y_true=0
    if(y_dev[i][1]==1):
        y_true=1
    y_pred = model.predict(image)
    y_pred_class = np.argmax(y_pred, axis = 1)[0]
    total += 1
    
    if y_pred_class == y_true:
            correct += 1        
    print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')
    print(f'Class: {class_names[y_true]}')
    print('******************')

In [ ]:
# fake_image = os.listdir('../input/casia-20-image-tampering-detection-dataset/CASIA2/Tp/')
# correct = 0
# total = 0
# for file_name in fake_image:
#     if file_name.endswith('jpg') or file_name.endswith('png'):
#         fake_image_path = os.path.join('../input/casia-20-image-tampering-detection-dataset/CASIA2/Tp/', file_name)
# #         image = prepare_image(fake_image_path)
#         image = ELA(fake_image_path)
#         image=image.resize((224, 224), PIL.Image.ANTIALIAS)
#         image=np.array(image)
#         image = image.reshape(-1, 224, 224, 3)
#         y_pred = model.predict(image)
#         y_pred_class = np.argmax(y_pred, axis = 1)[0]
#         total += 1
#         if y_pred_class == 1:
#             correct += 1
#             print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
# print(f'Total: {total}, Correct: {correct}, Acc: {correct / total * 100.0}')

In [ ]:
# real_image = os.listdir('../input/casia-20-image-tampering-detection-dataset/CASIA2/Au/')
# correct_r = 0
# total_r = 0
# for file_name in real_image:
#     if file_name.endswith('jpg') or file_name.endswith('png'):
#         real_image_path = os.path.join('../input/casia-20-image-tampering-detection-dataset/CASIA2/Au/', file_name)
#         image = ELA(real_image_path)
#         image=image.resize((224, 224), PIL.Image.ANTIALIAS)
#         image=np.array(image)
#         image = image.reshape(-1, 224, 224, 3)
#         y_pred = model.predict(image)
#         y_pred_class = np.argmax(y_pred, axis = 1)[0]
#         total_r += 1
#         if y_pred_class == 0:
#             correct_r += 1
#             print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
# correct += correct_r
# total += total_r
# print(f'Total: {total_r}, Correct: {correct_r}, Acc: {correct_r / total_r * 100.0}')
# print(f'Total: {total}, Correct: {correct}, Acc: {correct / total * 100.0}')

In [ ]:
# model.save('/kaggle/working/model_casia_run2.h5')
# model.save_weights('/kaggle/working/model_casia_run1.h5')

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/kaggle/working/model_casia_run.h5")
print("Saved model to disk")